In [76]:
# pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


2022-11-14 08:19:11,610	ERROR worker.py:401 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::apply_list_of_funcs() (pid=8577, ip=169.255.255.2)
  File "/opt/conda/lib/python3.7/site-packages/modin/core/execution/ray/implementations/pandas_on_ray/partitioning/partition.py", line 479, in apply_list_of_funcs
    partition = func(partition, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/modin/core/dataframe/algebra/map.py", line 48, in <lambda>
    lambda x: function(x, *args, **kwargs), *call_args, **call_kwds
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py", line 8825, in applymap
    return self.apply(infer).__finalize__(self, "applymap")
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py", line 8740, in apply
    return op.apply()
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/apply.py", line 688, in apply
    return self.apply_standard()
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/

In [39]:
import warnings, os
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv('env')
import sqlite3
import pandas as pd
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
import spacy
from importlib import reload
from modules import DataCollection, Support
reload(Support)
reload(DataCollection)
from modules.Support import Corpus, Business, TokenCleaner

In [10]:
import ray, os
ray.init(num_cpus=32)
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as md

2022-11-14 06:47:57,417	INFO worker.py:1528 -- Started a local Ray instance.


In [38]:
business = md.read_csv('data/business.csv', compression = 'zip')
review = md.read_csv('data/review.csv', compression = 'zip')

In [69]:
review_text = review[['review_id', 'text']]

In [ ]:
tc = TokenCleaner()
review_text['text_cleaned'] = review_text['text'].apply(tc.CleanText)

In [34]:
text_tokenized = review_text[['review_id', 'text_cleaned']]
text_tokenized.head()

,review_id,text_cleaned
0,KU_O5udG6zpxOg-VcAEodg,"[decid, eat, awar, go, take, 2, hour, begin, e..."
1,saUsX_uimxRlCVr67Z4Jig,"[famili, diner, buffet, eclect, assort, larg, ..."
2,AqPFMleE6RsU23_auESxiA,"[wow, yummi, differ, delici, favorit, lamb, cu..."
3,Sx8TMOWLNuJBWer-0pcmoA,"[cute, interior, owner, gave, us, tour, upcom,..."
4,JrIxlS1TzJ-iCu79ul40cQ,"[long, term, frequent, custom, thi, establish,..."


In [54]:
db = sqlite3.connect('database.db')

In [55]:
review._to_pandas().to_sql('review', db, if_exists='replace', index=False)
db.commit()

In [56]:
business._to_pandas().to_sql('business', db, if_exists='replace', index=False)
db.commit()

In [57]:
text_tokenized._to_pandas().to_sql('text_tokenized', db, if_exists='replace', index=False)
db.commit()

In [72]:
s3_client = DataCollection.s3_client()
s3_client.upload_file('database.db', 's3fld', '599Team12/database.db')

In [29]:
# use itertools to count  all the words in the text_cleaned column
word_counts = Counter(chain.from_iterable(review_text['text_cleaned']))

# convert the word_counts dict to a dataframe and sort it by the count
word_counts = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
word_counts.columns = ['word', 'count']
word_counts = word_counts.sort_values(by='count', ascending=False)
word_counts.head()